In [25]:
import sys
import os

# خلي Python يشوف مجلد src
sys.path.append(os.path.abspath(os.path.join("..", "src")))
# دلوقتي تقدر تستدعي الموديولز
from features.distances import add_distance_features
from features.timeparts import add_time_features
from features.encoding import add_one_hot_encoding
from data.Data_helper import save_processed_data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [26]:
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")

print(df.shape)
print(df.head())
print(df.info())

(1458644, 11)
          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1        -73.979027        40.763939         -74.005333   
3                1        -74.010040        40.719971         -74.012268   
4                1        -73.973053        40.793209         -73.972923   

   dropoff_latitude store_and_fwd_flag  trip_duration  
0         40.765602                  N            455  
1 

دلوقتي خدنا فكره عامه عن الداتا بتاعتنا لقيناها مش معبره كفايه ولكن منها نقدر ننشأ features جديده تكون معبره اكتر وليها علاقه مباشره بالتارجت بتاعنا 
زي ايه بقا نجيب بعض المسافات من معلومات خطوط االطول ودواير العرض بتاع مكان الاقلاع ومكان الانتهاء 
Distances: Haversine, Manhattan
Directions: Bearing ,speed_kmh
وبرضو عمود الداتاتايم لو فصلناه يبقا افضل لان الساعه ممكن تكون اهم من التاريخ وهل اليوم ده كان ويك اند ولا عادي فبرضو هنعمل شغل علي اعمدة الداتا تايم 
كل ده هنستخدم ملف  ال features لان كل الدوال منظمه جواه

In [27]:
# ==== 9️⃣ Feature Ideas Generation ====

# 1. Distance-based features
from features.distances import add_distance_features
from features.timeparts import add_time_features

# Make sure df is defined
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")
df['trip_duration_min'] = df['trip_duration'] / 60

df = add_distance_features(df)

# 2. Time-based features (using pickup_datetime)
df = add_time_features(df, datetime_col="pickup_datetime")

print("\n Columns after add_distance_features:")
print(df.columns.tolist())

# 3. Quick check
print("\n✅ Features added successfully!")
print(df.info())
print(df.head(10)[[
    "trip_duration_min", 
    "haversine_km", "manhattan_km", "bearing_deg",
    "hour", "weekday", "month", "is_weekend", "is_rush_hour"
]])
print(df[["trip_duration_min", "haversine_km", "manhattan_km", "speed_kmh", "bearing_deg"]].head())

✅ add_distance_features: 1458644/1458644 rows got valid speed_kmh

 Columns after add_distance_features:
['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'weekday', 'month', 'is_weekend', 'is_rush_hour']

✅ Features added successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 21 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1458644 non-null  object        
 1   vendor_id           1458644 non-null  int64         
 2   pickup_datetime     1458644 non-null  datetime64[ns]
 3   dropoff_datetime    1458644 non-null  object        
 4   passenger_count     1458644 non-null  int64         
 5   pic

دلوقتي بعد ما ضيفنا الفيتشرز الجديده وشوفناا شويه منهم فعلا نقدر نقول انها منطقيه اكتر وليها معني 
المسافات الاكبر بتعمل وقت اعلي ولو مسافتين متساويتين السرعه الاعلي تعمل مده اقل 

الداتا عندنا بتاع التارجت فيها شوية  skewed 
Log Transformationيعني التوزيع بتاعها مش طبيعي اوي عشان كده رأيي نطبق عليه دالة ال 

In [28]:
df = df.drop(columns=["trip_duration"])


In [29]:
df["log_trip_duration_min"] = np.log1p(df["trip_duration_min"])
print (df.head(10))

          id  vendor_id     pickup_datetime     dropoff_datetime  \
0  id2875421          2 2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1 2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2 2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2 2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2 2016-03-26 13:30:55  2016-03-26 13:38:10   
5  id0801584          2 2016-01-30 22:01:40  2016-01-30 22:09:03   
6  id1813257          1 2016-06-17 22:34:59  2016-06-17 22:40:40   
7  id1324603          2 2016-05-21 07:54:58  2016-05-21 08:20:49   
8  id1301050          1 2016-05-27 23:12:23  2016-05-27 23:16:38   
9  id0012891          2 2016-03-10 21:45:01  2016-03-10 22:05:26   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1     

عندنا أعمدة في الداتا مش أرقام، او ارقام مترتبه
:
- `vendor_id`
- `store_and_fwd_flag`
- `hour`
- `weekday`
- `month`

الموديل ما بيعرفش يتعامل مع الحروف أو القيم اللي مش أرقام.  
ولو أرقام عادية (1، 2، 3 …) الموديل هيفتكر إن في ترتيب ومعنى للفرق بينهم، وده مش صح.

عشان كده بنستخدم **One-Hot Encoding**:
- بيحوّل كل قيمة كاتيجوري لعمود جديد فيه 0 و 1.
- بيمنع الموديل يفتكر إن في علاقة ترتيب بين القيم.
- بيسهّل على الموديل يتعلم صح.
- 
مثال:  
لو عندنا `vendor_id` (1 أو 2) → بعد الـ encoding هيتحوّل لعمود `vendor_id_2` فيه:
- 1 لو الفيندور = 2
- 0 لو الفيندور = 1  

وبكده نكون مثلنا الكاتيجوري بطريقة أوضح للموديل.

In [30]:
df = add_one_hot_encoding(df)
print(df.columns.tolist())

print(df.head(10))

['id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'is_weekend', 'is_rush_hour', 'log_trip_duration_min', 'vendor_id_1', 'vendor_id_2', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6']
          id     pickup_datetime     dropoff_datetime  passenger_count  \
0  id2875421 2016-03-14 17:24:55  2016-03-14 17:32:30                1   
1  id2377394 2016-06-12 00:43:35  2016-06-12 00:54:38                1   
2  id3858529 2016-01-19 11:35:24  2016-01-19 12:10:48                1   
3  id3504673 2016-04-06 19:32:31  2016-04-06 19:39:40                1   
4  id2181028 2016-03-26 13:30:55  2016-03-26 13:38:10                1   
5  id0801584 2016-01-30 22:01:40

 EDA نحفظ بقا الملف عشان نرجع نبص عالعلاقات فالنةتبةك بتاع ال 

In [31]:
save_processed_data(df, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv")


✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv


نرجع نبص بصه 

In [32]:
path = "G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed.csv"
df = pd.read_csv(path)

df.head()
print(df.shape)
print(df.info())
df.describe()

(1458644, 34)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 34 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   id                     1458644 non-null  object 
 1   pickup_datetime        1458644 non-null  object 
 2   dropoff_datetime       1458644 non-null  object 
 3   passenger_count        1458644 non-null  int64  
 4   pickup_longitude       1458644 non-null  float64
 5   pickup_latitude        1458644 non-null  float64
 6   dropoff_longitude      1458644 non-null  float64
 7   dropoff_latitude       1458644 non-null  float64
 8   trip_duration_min      1458644 non-null  float64
 9   haversine_km           1458644 non-null  float64
 10  manhattan_km           1458644 non-null  float64
 11  bearing_deg            1458644 non-null  float64
 12  speed_kmh              1458644 non-null  float64
 13  hour                   1458644 non-null  int64  
 14  is_w

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration_min,haversine_km,manhattan_km,bearing_deg,speed_kmh,hour,is_weekend,is_rush_hour,log_trip_duration_min
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,1.599154e+01,3.440864e+00,4.446650e+00,-1.566892e+01,1.442321e+01,1.360648e+01,2.853568e-01,4.290293e-01,2.488660e+00
std,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,8.729053e+01,4.296538e+00,5.659821e+00,1.044915e+02,1.497752e+01,6.399693e+00,4.515844e-01,4.949377e-01,6.953023e-01
min,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.666667e-02,0.000000e+00,0.000000e+00,-1.799927e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.652930e-02
25%,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,6.616667e+00,1.231837e+00,1.570939e+00,-1.252736e+02,9.121275e+00,9.000000e+00,0.000000e+00,0.000000e+00,2.030339e+00
50%,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,1.103333e+01,2.093717e+00,2.688802e+00,8.199996e+00,1.279198e+01,1.400000e+01,0.000000e+00,0.000000e+00,2.487681e+00
75%,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.791667e+01,3.875337e+00,4.998980e+00,5.340495e+01,1.784491e+01,1.900000e+01,1.000000e+00,1.000000e+00,2.940043e+00
max,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,5.877137e+04,1.240909e+03,1.336846e+03,1.800000e+02,9.274837e+03,2.300000e+01,1.000000e+00,1.000000e+00,1.098143e+01


لاحظت ان في قيم مش منطقيه زي ان في مدة رحله وصل ل 40 يوم فغالبا في اوتلايرز محتاجين نشوفها ونتعامل معاها


In [37]:
columns = ['trip_duration_min', 'haversine_km', 'manhattan_km', 'speed_kmh']

factor = 2  # ممكن تزوده لـ 2.5 لو عايز تقلل أكتر

for col in columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.2f}%)")

trip_duration_min: 47548 outliers (3.26%)
haversine_km: 109913 outliers (7.54%)
manhattan_km: 97426 outliers (6.68%)
speed_kmh: 32910 outliers (2.26%)


دي كده نسبة الامثله اللي فيها اوتلاير واضح 
من النسبه دي من رايي نسبه مش كبيره االافضل نمسحها

In [38]:
columns = ['trip_duration_min', 'haversine_km', 'manhattan_km', 'speed_kmh']

factor = 2  # تقدر تغيره لتقليل أو زيادة عدد القيم الشاذة

df_clean = df.copy()
original_rows = len(df_clean)

for col in columns:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - factor * IQR
    upper = Q3 + factor * IQR
    df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]

cleaned_rows = len(df_clean)
deleted_rows = original_rows - cleaned_rows
deleted_percent = deleted_rows / original_rows * 100



print(f"Original rows: {original_rows}")
print(f"Cleaned rows: {cleaned_rows}")
print(f"Deleted rows: {deleted_rows} ({deleted_percent:.2f}%)")

Original rows: 1458644
Cleaned rows: 1274142
Deleted rows: 184502 (12.65%)


In [39]:
df_clean.head()
print(df_clean.shape)
print(df_clean.info())
df_clean.describe()

(1274142, 34)
<class 'pandas.core.frame.DataFrame'>
Index: 1274142 entries, 0 to 1458643
Data columns (total 34 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   id                     1274142 non-null  object 
 1   pickup_datetime        1274142 non-null  object 
 2   dropoff_datetime       1274142 non-null  object 
 3   passenger_count        1274142 non-null  int64  
 4   pickup_longitude       1274142 non-null  float64
 5   pickup_latitude        1274142 non-null  float64
 6   dropoff_longitude      1274142 non-null  float64
 7   dropoff_latitude       1274142 non-null  float64
 8   trip_duration_min      1274142 non-null  float64
 9   haversine_km           1274142 non-null  float64
 10  manhattan_km           1274142 non-null  float64
 11  bearing_deg            1274142 non-null  float64
 12  speed_kmh              1274142 non-null  float64
 13  hour                   1274142 non-null  int64  
 14  is_weeken

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration_min,haversine_km,manhattan_km,bearing_deg,speed_kmh,hour,is_weekend,is_rush_hour,log_trip_duration_min
count,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06,1.274142e+06
mean,1.660824e+00,-7.397970e+01,4.075289e+01,-7.397803e+01,4.075285e+01,1.153257e+01,2.294081e+00,2.944654e+00,-1.641785e+01,1.277192e+01,1.372009e+01,2.844981e-01,4.402500e-01,2.369969e+00
std,1.312443e+00,6.686709e-02,2.653352e-02,6.726901e-02,2.861175e-02,7.006151e+00,1.545452e+00,1.987255e+00,1.053279e+02,5.680967e+00,6.292898e+00,4.511752e-01,4.964173e-01,5.830919e-01
min,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.435970e+01,1.666667e-02,0.000000e+00,0.000000e+00,-1.799918e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.652930e-02
25%,1.000000e+00,-7.399219e+01,4.073867e+01,-7.399169e+01,4.073799e+01,6.250000e+00,1.152391e+00,1.465127e+00,-1.300189e+02,8.724849e+00,9.000000e+00,0.000000e+00,0.000000e+00,1.981001e+00
50%,1.000000e+00,-7.398223e+01,4.075422e+01,-7.398093e+01,4.075460e+01,1.005000e+01,1.863801e+00,2.390178e+00,7.603510e+00,1.198077e+01,1.400000e+01,0.000000e+00,0.000000e+00,2.402430e+00
75%,2.000000e+00,-7.396983e+01,4.076767e+01,-7.396711e+01,4.076873e+01,1.530000e+01,3.039295e+00,3.915585e+00,5.224830e+01,1.607451e+01,1.900000e+01,1.000000e+00,1.000000e+00,2.791165e+00
max,9.000000e+00,-6.133553e+01,4.391176e+01,-6.133553e+01,4.391176e+01,4.051667e+01,8.427831e+00,9.295454e+00,1.800000e+02,3.144230e+01,2.300000e+01,1.000000e+00,1.000000e+00,3.726095e+00


In [40]:
save_processed_data(df_clean, r"G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed_filtered.csv")


✅ Processed data saved to G:/ML mostafa saad/slides/my work/13 Project 1 - Regression - Trip Duration Prediction/data/processed/train_processed_filtered.csv
